In [244]:
from Bio import SeqIO
import numpy as np
from PIL import Image
from skimage.filters import gaussian
import pandas as pd
import Polywarp
import skimage.io as io
import matplotlib.pyplot as plt
from skimage.feature import blob_log, blob_doh, blob_dog
from scipy import ndimage
from skimage.feature import peak_local_max
from scipy import spatial
from cmath import inf
from skimage import transform
from sklearn.metrics import mean_squared_error
from numpy.linalg import norm
from math import floor, ceil

In [245]:
def residual_cal(src, dst):
    """This function is used to calculate the residual between src and dst.

    Args:
        src (numpy array): The source array having (N, 2) shape with N points.
        dst (numpy array): The destination array having (N, 2) shape with N points.

    Returns:
        mse: The mean squared error between these two arraies.
    """
    mse = mean_squared_error(src, dst)
    return mse
def count_nearest_pts(src, dst, radius):
    """Counting the number of nearest neighbors for each given point.

    Args:
        src (numpy array): (N, 2) shape array. Build the kd tree based on this.
        dst (numpy array): (N, 2) shape array. For each point in this array, find the nearest neighbors in src array.
        radius (int): The maximum searching radius.

    Returns:
        res, idx: res is the distance for the point and its neighbor, 'inf' means no neighbor in given search radius. 
        idx is the index for the neighbor in src array.
    """
    tree = spatial.KDTree(src)
    res, idx = tree.query(dst, k=1, distance_upper_bound=radius)
    for i in range(0, len(idx)):
        if len(np.argwhere(idx == idx[i])) > 1:
            res[i] = inf
    return res, idx

def blob_detection(img, min_sigma, max_sigma, threshold, method=0):
    """This function is mostly used for detecting the beads in any image.

    Args:
        img_path (string): The absolute path of the input image.
        min_sigma (int): The minimum sigma, lower it is, smaller the blob will be detected.
        max_sigma (int): The maximum sigma, higher it is, bigger the blob will be detected.
        threshold (float): Higher it is, higher the intensities of blobs.
        method (int, optional): 0 for Difference of Gaussian (DoG) and 1 for Determinant of Hessian (DoH). 
        They should be applied with different combination of parameters. DoG is more suitable for fret movies,
        while DoH is more suitable for sequencing images. Defaults to 0.

    Returns:
        centers: A numpy array containing the coordinates of all the centers.
    """
    #img = io.imread(img_path)
    if method == 2: #Local maxima
        coordinates = peak_local_max(img, min_distance=5)
        centers = []
        h, w = img.shape
        r = 3
        for i in range(coordinates.shape[0]):
            y, x = coordinates[i]
            if y > r and y < (h - r) and x > r and x < (w - r):
                centers.append(
                    ndimage.measurements.center_of_mass(
                        img[int(y - r) : int(y + r + 1), int(x - r) : int(x + r + 1)]
                    )
                )
                centers[i] = list(np.add(centers[i], [x - r, y - r]))
                i += 1
        
    else:
    
        if method == 0:
            blob = blob_dog(
                img, min_sigma=min_sigma, max_sigma=max_sigma, threshold=threshold
            )
        else:
            blob = blob_doh(
                img, min_sigma=min_sigma, max_sigma=max_sigma, threshold=threshold
            )
#         i = 0
        r = 3
        centers = []
        h, w = img.shape
        for blob in blob:
            y, x = blob
            if y > r and y < (h - r) and x > r and x < (w - r):
                centers.append(
                    ndimage.measurements.center_of_mass(
                        img[int(y - r) : int(y + r + 1), int(x - r) : int(x + r + 1)]
                    )
                )
                centers[i] = list(np.add(centers[i], [x - r, y - r]))
#                 i += 1
    return np.array(centers)

def show_blob_detection_res(img, centers):
    """
    Showing the result of 'blob detection' function. Used as the same way of 'blob_detection'
    """
    fig, ax = plt.subplots()
    #img = io.imread(img_path)
    ax.imshow(img)
    
    
    r = 3
    [h, w] = img.shape
    for i in range(centers.shape[0]):
        x, y = centers[i,:]
        # print(r)

        c = plt.Circle([x, y], 3, color="red", linewidth=1, fill=False)
        ax.add_patch(c)
        i += 1
    ax.set_axis_off()
    plt.show()

In [246]:
def get_pos(record):
    des = record.description
    tile_num = int(des.split(' ')[0].split(':')[4])
    x_pos = int(des.split(' ')[0].split(':')[5])
    y_pos = int(des.split(' ')[0].split(':')[6])
    return tile_num, x_pos, y_pos


def generate_img(x, y, x_min, y_min, x_max, y_max, op_path, r, blurred, sigma):
    # x_min = min(x)
    # x_max = max(x)
    # y_min = min(y)
    # y_max = max(y)
    x_range = x_max - x_min + 1
    y_range = y_max - y_min + 1
    # x_range = 27994
    # y_range = 27174
    img = np.zeros(shape=(x_range, y_range))
    for i in range(0, len(x)):
        img[max(0,int(x[i])-r):min(x_range,int(x[i]+r+1)),max(0,int(y[i])-r):min(y_range,int(y[i]+r+1))] = 200

    if blurred:
        img = gaussian(img, sigma=sigma)
    im = Image.fromarray(img)
    new_im = im.convert("L")
    new_im.save(op_path)


def generate_with_marker(x, y, marker_x, marker_y, op_path, csv_path):
    x_min = min(x)
    x_max = max(x)
    y_min = min(y)
    y_max = max(y)
    x_range = x_max - x_min + 1
    y_range = y_max - y_min + 1
    img = np.zeros(shape=(x_range, y_range, 3))
    marker_dic = {'index': range(0, len(marker_x)), 'x': [], 'y': []}
    for i in range(0, len(x)):
        for j in range(0, 5):
            for k in range(0, 5):
                img[max(0, x[i] - x_min - j),
                    max(0, y[i] - y_min - k), 0:3] = 255
                img[min(x_range - 1, x[i] - x_min + j),
                    max(0, y[i] - y_min - k), 0:3] = 255
                img[min(x_range - 1, x[i] - x_min + j),
                    min(y_range - 1, y[i] - y_min + k), 0:3] = 255
                img[max(0, x[i] - x_min - j),
                    min(y_range - 1, y[i] - y_min + k), 0:3] = 255
    for i in range(0, len(marker_x)):
        marker_dic['x'].append(marker_x[i] - x_min)
        marker_dic['y'].append(marker_y[i] - y_min)
        for j in range(0, 5):
            for k in range(0, 5):
                img[max(0, marker_x[i] - x_min - j),
                    max(0, marker_y[i] - y_min - k), 0] = 255
                img[max(0, marker_x[i] - x_min - j),
                    max(0, marker_y[i] - y_min - k), 1:3] = 0
                img[min(x_range - 1, marker_x[i] - x_min + j),
                    max(0, marker_y[i] - y_min - k), 0] = 255
                img[min(x_range - 1, marker_x[i] - x_min + j),
                    max(0, marker_y[i] - y_min - k), 1:3] = 0
                img[min(x_range - 1, marker_x[i] - x_min + j),
                    min(y_range - 1, marker_y[i] - y_min + k), 0] = 255
                img[min(x_range - 1, marker_x[i] - x_min + j),
                    min(y_range - 1, marker_y[i] - y_min + k), 1:3] = 0
                img[max(0, marker_x[i] - x_min - j),
                    min(y_range - 1, marker_y[i] - y_min + k), 0] = 255
                img[max(0, marker_x[i] - x_min - j),
                    min(y_range - 1, marker_y[i] - y_min + k), 1:3] = 0
    blurred_img = gaussian(img, sigma=3, multichannel=True)
    im = Image.fromarray(blurred_img.astype(np.uint8))
    new_im = im.convert("P")
    new_im.save(op_path)
    df = pd.DataFrame(marker_dic)
    df.to_csv(csv_path, index=True, header=True)


def get_coordinates(fastq_path):
    x_coordinate_01 = []
    y_coordinate_01 = []
    x_coordinate_02 = []
    y_coordinate_02 = []
    for record in SeqIO.parse(fastq_path, "fastq"):
        if record is not None:
            tile_num, x_pos, y_pos = get_pos(record)
            seq = str(record.seq)
            if tile_num == 1101:
                x_coordinate_01.append(x_pos)
                y_coordinate_01.append(y_pos)
            elif tile_num == 1102:
                x_coordinate_02.append(x_pos)
                y_coordinate_02.append(y_pos)
    print('Coordinates are found.')
    return x_coordinate_01, y_coordinate_01, x_coordinate_02, y_coordinate_02


def get_xy_coordinates(fastq_path, with_seq):
    x_coordinate_01 = []
    y_coordinate_01 = []
    x_coordinate_02 = []
    y_coordinate_02 = []
    x_seq_coordinate_01 = []
    y_seq_coordinate_01 = []
    x_seq_coordinate_02 = []
    y_seq_coordinate_02 = []
    for record in SeqIO.parse(fastq_path, "fastq"):
        if record is not None:
            tile_num, x_pos, y_pos = get_pos(record)
            seq = str(record.seq)
            if tile_num == 1101:
                if with_seq != seq:
                    x_coordinate_01.append(x_pos)
                    y_coordinate_01.append(y_pos)
                else:
                    x_seq_coordinate_01.append(x_pos)
                    y_seq_coordinate_01.append(y_pos)
            elif tile_num == 1102:
                if with_seq != seq:
                    x_coordinate_02.append(x_pos)
                    y_coordinate_02.append(y_pos)
                else:
                    x_seq_coordinate_02.append(x_pos)
                    y_seq_coordinate_02.append(y_pos)
    print('Coordinates are found.')
    return x_coordinate_01, y_coordinate_01, \
        x_coordinate_02, y_coordinate_02, \
        x_seq_coordinate_01, y_seq_coordinate_01, \
        x_seq_coordinate_02, y_seq_coordinate_02


def get_seq_coordinates(fastq_path, with_seq_1, with_seq_2):
    x_coordinate_01 = []
    y_coordinate_01 = []
    x_coordinate_02 = []
    y_coordinate_02 = []
    max_x = 0
    max_y = 0
    min_x = 9e7
    min_y = 9e7
    for record in SeqIO.parse(fastq_path, "fastq"):
        if record is not None:
            tile_num, x_pos, y_pos = get_pos(record)
            if x_pos > max_x:
                max_x = x_pos
            if x_pos < min_x:
                min_x = x_pos
            if y_pos > max_y:
                max_y = y_pos
            if y_pos < min_y:
                min_y = y_pos
            seq = str(record.seq)
            if tile_num == 1101:
                if with_seq_1 in seq or with_seq_2 in seq:
                    x_coordinate_01.append(x_pos)
                    y_coordinate_01.append(y_pos)
            elif tile_num == 1102:
                if with_seq_1 in seq or with_seq_2 in seq:
                    x_coordinate_02.append(x_pos)
                    y_coordinate_02.append(y_pos)
    print('Coordinates are found.')
    return x_coordinate_01, y_coordinate_01, x_coordinate_02, y_coordinate_02, max_x, max_y





# def get_seq_coordinates(fastq_path, lib_seq):
#     x_coordinate_01 = []
#     y_coordinate_01 = []
#     x_coordinate_02 = []
#     y_coordinate_02 = []
#     max_x_1 = 0
#     max_y_1 = 0
#     max_x_2 = 0
#     max_y_2 = 0
    
#     for record in SeqIO.parse(fastq_path, "fastq"):
#         if record is not None:
#             tile_num, x_pos, y_pos = get_pos(record)
#             seq = str(record.seq)
#             if tile_num == 1101:
#                 if 
#                 if lib_seq in seq:
#                     x_coordinate_01.append(x_pos)
#                     y_coordinate_01.append(y_pos)
#             elif tile_num == 1102:
#                 if lib_seq in seq:
#                     x_coordinate_02.append(x_pos)
#                     y_coordinate_02.append(y_pos)
#     print('Coordinates are found.')
#     return x_coordinate_01, y_coordinate_01, x_coordinate_02, y_coordinate_02


In [14]:
print(img[2:2,2:2])

[]


In [14]:
# Test
fastq_path = '/Users/qinhanhou/Desktop/DeindlLab/220722/SeqRes/donuts_1/Donuts1/0722.fastq'
x_coordinate_01, y_coordinate_01, x_coordinate_02, y_coordinate_02 \
    = get_seq_coordinates(fastq_path, 'A', 'qwwdasdqwraszdqwqw')
x_coordinate_01 = x_coordinate_01[10000:11000]
y_coordinate_01 = y_coordinate_01[10000:11000]


generate_img(x_coordinate_01, y_coordinate_01,
             '/Users/qinhanhou/Desktop/DeindlLab/220722/SeqImg/test.png', 1, False, 4)

Coordinates are found.


In [77]:
def get_seq_coordinates_v2(fastq_path, with_seq, tile):
    x_coordinate = []
    y_coordinate = []
    max_x = 0
    max_y = 0
    min_x = 9e7
    min_y = 9e7
    for record in SeqIO.parse(fastq_path, "fastq"):
        if record is not None:
            tile_num, x_pos, y_pos = get_pos(record)
            x_pos = x_pos/10
            y_pos = y_pos/10

            seq = str(record.seq)
            if tile_num == tile:
                if x_pos > max_x:
                    max_x = x_pos
                if x_pos < min_x:
                    min_x = x_pos
                if y_pos > max_y:
                    max_y = y_pos
                if y_pos < min_y:
                    min_y = y_pos
                if with_seq in seq:
                    x_coordinate.append(x_pos)
                    y_coordinate.append(y_pos)
    print('Coordinates are found.')
    return x_coordinate, y_coordinate, max_x, max_y, min_x, min_y

# def generate_img_v2(x, y, x_min, y_min, x_max, y_max, op_path, r, blurred, sigma):
    

In [83]:
import tkinter as tk
from tkinter import filedialog
from math import ceil
# root = tk.Tk()
# root.attributes("-topmost", True)
# root.withdraw()

fastq_path = filedialog.askopenfilename(title = "Choose the FASTQ file", initialdir = "D:/Experiments/MUSCLE DONUTS/20220824_FC_Nano_Cas9_CCR5/")

# x_coordinate_01m, y_coordinate_01m, x_coordinate_02m, y_coordinate_02m, max_x, max_y, min_x, min_y \
#     = get_seq_coordinates(fastq_path, 'GGTCTCGTCCAATCTAT', 'qwwdasdqwraszdqwqw')
# x_coordinate_01, y_coordinate_01, x_coordinate_02, y_coordinate_02 \
#     = get_seq_coordinates(fastq_path, 'A', 'qwwdasdqwraszdqwqw')
x_coord, y_coord, max_x, max_y, min_x, min_y = get_seq_coordinates_v2(fastq_path=fastq_path, with_seq='A', tile=1101)
# print(len(x_coordinate_01))
# print(len(x_coordinate_02))
# print(np.max(x_coordinate_01))
# print(np.min(x_coordinate_01))
x_coord1 = np.subtract(max_y, y_coord)
y_coord = np.subtract(max_x, x_coord)
x_coord = x_coord1
max_x1 = ceil(max_y)
max_y = ceil(max_x)
max_x = max_x1
# max_x = ceil(max_x)
# max_y = ceil(max_y)
print(max_x)
print(min_x)
print(max_y)
print(min_y)
print(max_x - min_x)
# generate_with_marker(x_coordinate_01, y_coordinate_01, x_coordinate_01m, y_coordinate_01m,
#   '/Users/qinhanhou/Desktop/DeindlLab/220722/SeqImg/1101_marked_sigma3.png',
#   '/Users/qinhanhou/Desktop/DeindlLab/220722/SeqImg/1101_marked_sigma3.csv')
generate_img(y_coord,x_coord, 0, 0, max_y, max_x,
             'D:/Experiments/MUSCLE DONUTS/220719/FASTQ_image.png', 1, True, 1)
# generate_img(x_coordinate_02, y_coordinate_02,
            #  '/Users/qinhanhou/Desktop/DeindlLab/220722/SeqImg/1102_full_sigma3.png', 3, True, 4)


Coordinates are found.
2885
172.3
2972
133.3
2712.7


In [81]:
print(fastq_path)

D:/Experiments/MUSCLE DONUTS/220719/0714.fastq


In [91]:
x_coord_cor = np.add(x_coord,19.815)
y_coord_cor = np.add(y_coord,27.14)
# generate_img(np.add(y_coord,27.14),np.add(x_coord,19.815), 0, 0, 2943,2865,
#              'D:/Experiments/MUSCLE DONUTS/220719/FASTQ_image.png', 1, True, 1)

In [15]:
seq_path = filedialog.askopenfilename(title = "Choose the seq file", initialdir = "D:/Experiments/MUSCLE DONUTS/220719/")
seq_img = io.imread(seq_path)

In [92]:

seq_centers = blob_detection(
    seq_img,
    min_sigma=1,
    max_sigma=5,
    threshold=0.001,
    method=2
)

# show_blob_detection_res(
#     seq_img,
#     seq_centers)

In [95]:

FQ_centers_ori = np.c_[x_coord,y_coord]
FQ_centers = np.c_[x_coord_cor,y_coord_cor]

In [96]:
print(FQ_centers[10])

[2769.215 1483.54 ]


In [93]:
max_x3 = 2865
max_y3 = 2943
# seq_centers = np.add(seq_centers, 0.5)

generate_img(seq_centers[:,1], seq_centers[:,0], 0, 0, max_y3, max_x3,
             'D:/Experiments/MUSCLE DONUTS/220719/Clusters_1.png', 1, True, 1)

6


In [101]:
# FQ_centers = tr(np.append(x_coord,y_coord, axis = 1))
res, idx = count_nearest_pts(FQ_centers, seq_centers, 2)
movie_centers1 = FQ_centers_ori[idx[np.where(res != inf)]]
seq_centers1 = seq_centers[np.where(res != inf)]
print(seq_centers1.shape[0], 'of', len(x_coord))

18748 of 238665


In [134]:
x1 = 500
x2 = 850
y1 = 400
y2 = 600
x_seq = seq_centers[:,0]
y_seq = seq_centers[:,1]
# np.where(x_seq>=x1)
seq_centers_cropped = seq_centers[np.where((x_seq<=x2)&(x_seq>=x1)&(y_seq<=y2)&(y_seq>=y1))]

In [142]:
src = seq_centers_cropped
dst = FQ_centers_ori
print(src.shape,dst.shape)

(510, 2) (238665, 2)


In [197]:
np.where(seq_centers[:,1]>100)
    

(array([    0,     1,     2, ..., 60248, 60249, 60250], dtype=int64),)

In [186]:
np.save('D:/Experiments/MUSCLE DONUTS/220719/src_temp.npy',src)

np.save('D:/Experiments/MUSCLE DONUTS/220719/dst_temp.npy',dst)

In [199]:
# src is a cropped cluster list, dst is all FASTQ coordinates
x_s = src[:,0]
y_s = src[:,1]
x_d = dst[:,0]
y_d = dst[:,1]

min_x_s = floor(min(x_s))
max_x_s = ceil(max(x_s))
min_y_s = floor(min(y_s))
max_y_s = ceil(max(y_s))
w_s = max_y_s - min_y_s
h_s = max_x_s - min_x_s

min_x_d = floor(min(x_d))
max_x_d = ceil(max(x_d))
min_y_d = floor(min(y_d))
max_y_d = ceil(max(y_d))
w_d = max_y_d - min_y_d
h_d = max_x_d - min_x_d

dx = min_x_d - min_x_s
dy = min_y_d - min_y_s
src1 = np.add(src, [dx,dy])
w_score = w_d-w_s
h_score = h_d-h_s

# score = np.zeros([w_score,h_score])
score = np.zeros([100,100])

radius = 2
it = np.nditer(score, flags=['multi_index'], op_flags=['writeonly'])

for x in it:
    dx, dy = it.multi_index
    x1 = min_x_d + dx
    y1 = min_y_d + dy
    x2 = x1 + h_s
    y2 = y1 + w_s
    
    dst1 = dst[np.where((dst[:,0]<=x2)&(dst[:,0]>=x1)&(dst[:,1]<=y2)&(dst[:,1]>=y1))]
    tree = spatial.KDTree(dst1)
    src2 = np.add(src1,it.multi_index)
    res, idx = tree.query(src2, k=1, distance_upper_bound=radius)
#     for i in range(0, len(idx)):
#         if len(np.argwhere(idx == idx[i])) > 1:
#             res[i] = inf
#     src2 = src2[np.where(res != inf)]
#     dst2 = dst[idx[np.where(res != inf)]]
#     x[...] = radius*src2.shape[0]-sum(norm(np.subtract(src2,dst2),axis=1))

# # it = np.nditer(a, flags=['multi_index'])

# for x in it:
#     print("%d <%s>" % (x, it.multi_index), end=' ')
print(score) 

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [224]:
import open3d as o3d
initial_T = np.identity(4)
length = src.shape[0]
src3d = np.concatenate((src,np.zeros([length,1])),axis = 1)
src_cloud = o3d.geometry.PointCloud()
src_cloud.points = o3d.utility.Vector3dVector(src3d)
distance = 3

length = dst.shape[0]
dst3d = np.concatenate((dst,np.zeros([length,1])),axis = 1)
dst_cloud = o3d.geometry.PointCloud()
dst_cloud.points = o3d.utility.Vector3dVector(dst3d)

# Define the type of registration:
type = o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
# "False" means rigid transformation, scale = 1

# Define the number of iterations (I'll use 100):
iterations = o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration = 100)

# Do the registration:
result = o3d.pipelines.registration.registration_icp(src_cloud, dst_cloud, distance, initial_T, type, iterations)


In [237]:
res, idx = count_nearest_pts(dst, src, 2)
srct =dst[np.where(res != inf)] 
print(srct.shape[0])
res, idx = count_nearest_pts(dst,np.add(src,[-5.5, 6.7]), 2)
srct =dst[np.where(res != inf)] 
print(srct.shape[0])


233
213


In [ ]:
src_cloud.points = o3d.utility.Vector3dVector(np.add(src3d,[-5.5,6.7,0]))
src_cloud.colors = o3d.utility.Vector3dVector(np.add(np.zeros(src3d.shape),[255,0,0]))
o3d.visualization.draw_geometries([src_cloud,dst_cloud])

In [189]:
tree = spatial.KDTree(dst)
res, idx = tree.query(src2, k=1, distance_upper_bound=radius)

In [163]:
print(max(norm(np.subtract(src1,dst1),axis=1)))


1.9904729531762613


In [122]:
FQ_centers = tr(FQ_centers_ori)
res, idx = count_nearest_pts(FQ_centers, seq_centers, 4)
movie_centers1 = FQ_centers_ori[idx[np.where(res != inf)]]
seq_centers1 = seq_centers[np.where(res != inf)]
print(seq_centers1.shape[0], 'of', len(x_coord))


order = 3
kx,ky = Polywarp.polywarp(seq_centers1[:,0],seq_centers1[:,1],movie_centers1[:,0],movie_centers1[:,1],degree=order)
print(kx)
print(ky)
tr = transform.PolynomialTransform()
#tr.estimate(src,dst,order = 2)

order1 = 2*order
pidx = 0
par = np.zeros([2,int((order1+1)*(order1+2)/2)])
for j in range(order1 + 1):
    for i in range(j + 1):
        if (j-i)<=order and i<=order:
            par[0, pidx] = kx[j - i,i]
            par[1, pidx] = ky[j - i,i]
        else:
            par[0, pidx] = 0
            par[1, pidx] = 0
        pidx += 1
tr.params = par        

# kx,ky = Polywarp.polywarp(movie_centers1[:,0],movie_centers1[:,1],seq_centers1[:,0],seq_centers1[:,1],degree=order)
# tr_inv = transform.PolynomialTransform()
# #tr.estimate(src,dst,order = 2)

# order1 = 2*order
# pidx = 0
# par = np.zeros([2,int((order1+1)*(order1+2)/2)])
# for j in range(order1 + 1):
#     for i in range(j + 1):
#         if (j-i)<=order and i<=order:
#             par[0, pidx] = kx[j - i,i]
#             par[1, pidx] = ky[j - i,i]
#         else:
#             par[0, pidx] = 0
#             par[1, pidx] = 0
#         pidx += 1
# tr_inv.params = par    

print('Residuals after poly (order = '+str(order)+') transformation: ',residual_cal(tr(movie_centers1), seq_centers1))
print('Residuals before transformation: ',residual_cal(movie_centers1, seq_centers1))

38859 of 238665
[[ 1.87990517e+01  4.42848180e-04  1.38004690e-06 -7.49012658e-10]
 [ 1.00086460e+00 -3.51791821e-07 -2.36607572e-09  1.35363150e-12]
 [ 2.03006046e-06 -6.32434994e-09  6.32604768e-12 -1.91244407e-15]
 [-1.12720731e-09  3.15110134e-12 -2.70008445e-15  7.05863419e-19]]
[[ 2.89151130e+01  9.98907047e-01 -2.37332801e-06  8.95351204e-10]
 [-8.34605080e-04 -1.22603078e-05  2.09802786e-08 -6.10987526e-12]
 [-8.64258103e-07  1.50746387e-08 -2.13880377e-11  6.00164191e-15]
 [ 3.21906655e-10 -3.98928311e-12  5.37321992e-15 -1.48433631e-18]]
Residuals after poly (order = 3) transformation:  2.5935181124131796
Residuals before transformation:  565.467516026154
